<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this lab, I'm going to solve the Lab: Segmenting and Clustering Neighborhoods in Toronto that is part of the Week 3 in the Data Science Capstone Course.

In [1]:
#!pip install beautifulsoup4
#!pip install lxml
#!pip install html5lib

import pandas as pd
import numpy as np
import html5lib
import requests
from bs4 import BeautifulSoup


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_raw = pd.read_html(url)
df_raw[0]

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Converting the table extracted into Data Frame.

In [3]:
neighborhoods = pd.DataFrame(df_raw[0])
neighborhoods

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Answer Question 1

Eliminating 'Districts' with the value 'Not assigned'.

In [4]:
neighborhoods = neighborhoods[neighborhoods.District != 'Not assigned']
neighborhoods

,Postal Code,District,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Looking for unique values in 'Neighbourhood' Column.

In [6]:
neighborhoods['Neighbourhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
       'Fairview, H

Looking one more time for rows in the Column 'Neighbourhood' with the value 'Not assigned'.

In [7]:
neighborhoods.loc[neighborhoods['Neighbourhood'] == 'Not assigned']

,Postal Code,District,Neighbourhood


Getting the 'shape' of the table. 

In [8]:
neighborhoods.shape

(103, 3)

Getting the latitude and the longitude coordinates of each neighborhood.

In [9]:
postal_code = neighborhoods['Postal Code']
postal_code = pd.DataFrame(postal_code)
postal_code

,Postal Code
2,M3A
3,M4A
4,M5A
5,M6A
6,M7A
...,...
160,M8X
165,M4Y
168,M7Y
169,M8Y


In [ ]:
#!pip install geocoder
#import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

Using the csv file provided by the course instructors to find the longitude and latitude of each Postal Code.

In [10]:
!wget -q -O 'toronto_neighborhoods_locations.csv' http://cocl.us/Geospatial_data
toronto_locations = pd.read_csv('toronto_neighborhoods_locations.csv')
toronto_locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Answer Question 2

Merging the both tables: the table that contains the neighborhoods and the one who has the location data.

In [11]:
neighborhoods_locations = pd.merge(left=neighborhoods, right=toronto_locations, left_on='Postal Code', right_on='Postal Code')
neighborhoods_locations

,Postal Code,District,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
